In [8]:
TESTING_SUBSET = False  # ***** FOR TESTING ONLY !!!!!!!! ******

import sys
IN_COLAB = 'google.colab' in sys.modules

In [94]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from functools import partialmethod
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')

from l_utils import *
from model_utils import *
from fe_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

if IN_COLAB:
    class CFG:
        # MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = DIRNAME_PROCESSED_DATA + "/models/"
else:
    from cfg import *
    
# topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized.pq"
# contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized.pq"

topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_ml64_dist.pq"
contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_ml64_dist.pq"

env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [10]:
def progress_bar_control(show_progress_bars = True):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=(not show_progress_bars))

In [ ]:
auto_tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL)

In [ ]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

In [137]:
def tokenize_main(tokenizer, topics_df, contents_df, topics_tokenized_pq_file, contents_tokenized_pq_file, max_length, debug = False, show_progress_bars=True):
    
    progress_bar_control(show_progress_bars)
    
    tokenize_df_and_write_to_parquet(tokenizer, retreiver_topics_FE, topics_df, text_cols = ['combined'], col_prefix = "t_", max_length = max_length, tokenized_ds_name = topics_tokenized_pq_file, debug = debug, post_process_df = None)

    tokenize_df_and_write_to_parquet(tokenizer, retreiver_contents_FE, contents_df, text_cols = ['combined'], col_prefix = "c_", max_length = max_length, tokenized_ds_name = contents_tokenized_pq_file, debug = debug)


In [138]:
# if (not(os.path.exists(topics_tokenized_pq_file) and os.path.exists(contents_tokenized_pq_file))):
tokenize_main(auto_tokenizer, topics_df.copy(),  contents_df.copy(), topics_tokenized_pq_file, contents_tokenized_pq_file, max_length = 64)
topics_tokenized_df   = pd.read_parquet(topics_tokenized_pq_file)
contents_tokenized_df = pd.read_parquet(contents_tokenized_pq_file)

Map (num_proc=2):   0% 0/61517 [00:00<?, ? examples/s]

Map (num_proc=2):   2% 1000/61517 [00:00<00:45, 1322.69 examples/s]

Map (num_proc=2):   5% 3000/61517 [00:00<00:14, 3902.84 examples/s]

Map (num_proc=2):   8% 5000/61517 [00:01<00:09, 6030.90 examples/s]

Map (num_proc=2):  11% 7000/61517 [00:01<00:07, 7685.93 examples/s]

...

Map (num_proc=2):  96% 59000/61517 [00:05<00:00, 12889.48 examples/s]

Map (num_proc=2):  99% 60759/61517 [00:05<00:00, 13203.38 examples/s]

Creating parquet from Arrow format:   0% 0/62 [00:00<?, ?ba/s]

Creating parquet from Arrow format:  66% 41/62 [00:00<00:00, 409.08ba/s]

Creating parquet from Arrow format: 100% 62/62 [00:00<00:00, 410.45ba/s]

In [63]:
topics_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61517 entries, 0 to 61516
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  61517 non-null  object
 1   t_0_input_ids       61517 non-null  object
 2   t_1_attention_mask  61517 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [128]:
SEP_TOKEN = auto_tokenizer.sep_token
SEP_TOKEN_ENCODING = 2
UNK_TOKEN_ENCODING = 3
CLS_TOKEN_ENCODING = 0
def count_sep(l):
    count = 0
    length = len(l)
    for i, item in enumerate(l):
        if item == SEP_TOKEN_ENCODING:
            if (i+1 < length and (l[i+1] == CLS_TOKEN_ENCODING)):
                count += 1
    return count
        

In [134]:
topics_tokenized_df['combined_sep_count'] = topics_tokenized_df['t_0_input_ids'].apply(lambda l: count_sep(l))
topics_tokenized_df['combined_sep_count'].value_counts()

3    42086
2    11779
1     7651
0        1
Name: combined_sep_count, dtype: int64

In [136]:
contents_tokenized_df['combined_sep_count'] = contents_tokenized_df['c_0_input_ids'].apply(lambda l: count_sep(l))
contents_tokenized_df['combined_sep_count'].value_counts()

3    76045
2    65852
1    12147
0        3
Name: combined_sep_count, dtype: int64

In [65]:
# topics_tokenized_df[['t_0_input_ids']][0:10]
l = list(topics_df[['title']][0:10].values.tolist())

In [66]:
l = flatten_l(l)
l

['Откриването на резисторите',
 'Unit 3.3 Enlargements and Similarities',
 'Entradas e saídas de uma função',
 'Transcripts',
 'Графики на експоненциални функции (Алгебра 2 ниво)',
 '100 સુધીનો સરવાળો',
 '12. 20: Bird Reproduction',
 'Nepal',
 '2.1.2 - Logarithms',
 'As vacinas']

In [67]:
encoded_input = auto_tokenizer(l, padding=True, truncation=True, return_tensors='pt')

In [68]:
encoded_input['input_ids'][0]

tensor([     0,   3858,  18229,  16317,     29,      6, 150652,  61056,    226,
             2,      1,      1,      1,      1,      1,      1,      1])

In [78]:
auto_tokenizer

PreTrainedTokenizerFast(name_or_path='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})